# Setting up the Notebook

## Dependencies
Loading in dependencies for the project.
- **pandas**: Used for DataFrame wrangling
- **os**: Used to define the filepath to load in data
- **datetime**: Used for date/time conversions
- **numpy**: Used for randomization and other
- **sklearn tree**: Decision Tree Classifier function
- **sklearn.ensemble RandomForestClassifier**: Random Forest Classifier function

In [1]:
import pandas as pd
import os
import numpy as np
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

### Global Seed
We will define the global seed that we will later use to randomly select training and testing groups.  
We used *61518* but you can use whatever number you would like to vary your results.

In [2]:
global_seed = 61518

In [3]:
np.random.seed(global_seed)

## Load in our Machine Learning DataFrame
We have a DataFrame that was built specifically for machine learning and other analytics. 
We load that in here.

In [4]:
file_to_load1 = os.path.join("Output Data", "ml_df.csv")
ml_df = pd.read_csv(file_to_load1)

# Machine Learning
Now that we have all necessary fields in a passable format, we will look to classify **Outcome** using a Decision Tree classifier.

## Create our training and testing groups
To avoid having any one **User**'s game results used to predict their other game results, we will be randomly selecting 5 **User**s as the testing group.  
This will cause there to be an imbalance between training and testing data, straying from a 4:1 ratio and between **Crewmate** and **Imposter** analysis. Given the explicit problem caused otherwise, we will use this method until a more appropriate method is determined.

### Create a list of users
We will use this list to split our data into a training and test groups

In [5]:
user_list = np.arange(1, ml_df["User"].astype(int).max() + 1, 1).tolist()
user_list

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29]

### Define groups
We will define our test group first, specifying without replacement to avoid repetition.  
We then define our training group by what is not in our test group.

In [6]:
test_list = np.random.choice(user_list, 5, replace = False)
test_list

array([24, 10, 27, 19, 26])

In [7]:
train_list = []
for x in user_list:
    if x not in test_list:
        train_list.append(x)
train_list = np.array(train_list)
train_list

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 11, 12, 13, 14, 15, 16, 17, 18,
       20, 21, 22, 23, 25, 28, 29])

Using our test and training lists, we split our data.

In [8]:
test_group = ml_df.loc[ml_df["User"].astype(int).isin(test_list)]
train_group = ml_df.loc[ml_df["User"].astype(int).isin(train_list)]

Split our groups between crewmates and imposters and drop unnecessary columns.

In [9]:
crewmate_test_df = test_group.loc[test_group["Team"] == 1].reset_index(drop=True)
crewmate_test_df.drop(columns = ["Imposter Kills"], inplace = True)
crewmate_train_df = train_group.loc[train_group["Team"] == 1].reset_index(drop=True)
crewmate_train_df.drop(columns = ["Imposter Kills"], inplace = True)

imposter_test_df = test_group.loc[test_group["Team"] == 0].reset_index(drop=True)
imposter_test_df.drop(columns = ["Task Completed", "All Tasks Completed", "Sabotages Fixed", "Time to complete all tasks", "Murdered"], inplace = True)
imposter_train_df = train_group.loc[train_group["Team"] == 0].reset_index(drop=True)
imposter_train_df.drop(columns = ["Task Completed", "All Tasks Completed", "Sabotages Fixed", "Time to complete all tasks", "Murdered"], inplace = True)

In [10]:
crewmate_train_df.columns

Index(['User', 'Team', 'Outcome', 'Game Length', 'Task Completed',
       'All Tasks Completed', 'Time to complete all tasks', 'Sabotages Fixed',
       'Murdered', 'Ejected', 'Night', 'Morning', 'Afternoon', 'Evening'],
      dtype='object')

In [11]:
imposter_train_df.columns

Index(['User', 'Team', 'Outcome', 'Game Length', 'Imposter Kills', 'Ejected',
       'Night', 'Morning', 'Afternoon', 'Evening'],
      dtype='object')

## Machine Learning - Imposter
We start by defining an appropriate model to predict wins or losses for **Imposter**s.

### Define Model #1
We will have to specify what it is that we are trying to predict and with what data.
- **Target**: Outcome
- **Data**: Game Length, Imposter Kills, Ejected, Night, Morning, Afternoon, Evening

In [12]:
target_imposter_train = imposter_train_df["Outcome"]
data_imposter_train = imposter_train_df[["Game Length", "Imposter Kills", "Ejected", "Afternoon", "Evening"]]

target_imposter_test = imposter_test_df["Outcome"]
data_imposter_test = imposter_test_df[["Game Length", "Imposter Kills", "Ejected", "Afternoon", "Evening"]]

features_imposter1 = data_imposter_train.columns

### Decision Tree Classification - Model #1
We perform a decision tree classification in order to assess the validity of our features.

In [13]:
dtc_imposter1 = tree.DecisionTreeClassifier()
dtc_imposter1 = dtc_imposter1.fit(data_imposter_train, target_imposter_train)
dtc_imposter1.score(data_imposter_test, target_imposter_test)

0.7714285714285715

In [14]:
sorted(zip(dtc_imposter1.feature_importances_, features_imposter1), reverse=True)

[(0.6181256375661297, 'Game Length'),
 (0.18408111394261287, 'Ejected'),
 (0.11110580110194608, 'Imposter Kills'),
 (0.043633143887246685, 'Afternoon'),
 (0.0430543035020647, 'Evening')]

In [15]:
dtc_predict_imposter1_df = pd.DataFrame({
    "Prediction_i" : dtc_imposter1.predict(data_imposter_test),
    "Actual_i" : target_imposter_test
})
dtc_predict_imposter1_df["Equals_i"] = dtc_predict_imposter1_df["Prediction_i"].eq(dtc_predict_imposter1_df["Actual_i"])
dtc_predict_imposter1_df

,Prediction_i,Actual_i,Equals_i
0,0,0,True
1,0,0,True
2,0,0,True
3,0,0,True
4,0,1,False
...,...,...,...
65,0,0,True
66,0,0,True
67,1,1,True
68,1,1,True


In [16]:
dtc_predict_imposter1_df["Equals_i"].value_counts()

True     54
False    16
Name: Equals_i, dtype: int64

### Random Forest Classification - Model #1
We perform a random forest classification in order to assess the validity of our features.

In [17]:
rfc_imposter1 = RandomForestClassifier()
rfc_imposter1 = rfc_imposter1.fit(data_imposter_train, target_imposter_train)
rfc_imposter1.score(data_imposter_test, target_imposter_test)

0.8428571428571429

In [18]:
sorted(zip(rfc_imposter1.feature_importances_, features_imposter1), reverse=True)

[(0.6564780047658015, 'Game Length'),
 (0.182189843519634, 'Ejected'),
 (0.11533834457199822, 'Imposter Kills'),
 (0.023685677635823556, 'Evening'),
 (0.02230812950674277, 'Afternoon')]

In [19]:
rfc_predict_imposter1_df = pd.DataFrame({
    "Prediction_i" : rfc_imposter1.predict(data_imposter_test),
    "Actual_i" : target_imposter_test
})
rfc_predict_imposter1_df["Equals_i"] = rfc_predict_imposter1_df["Prediction_i"].eq(rfc_predict_imposter1_df["Actual_i"])
rfc_predict_imposter1_df

,Prediction_i,Actual_i,Equals_i
0,0,0,True
1,0,0,True
2,0,0,True
3,0,0,True
4,1,1,True
...,...,...,...
65,0,0,True
66,0,0,True
67,1,1,True
68,1,1,True


In [20]:
rfc_predict_imposter1_df["Equals_i"].value_counts()

True     59
False    11
Name: Equals_i, dtype: int64

## Machine Learning - Crewmate
We start by defining appropriate models to predict wins or losses for **Crewmate**s.

### Define Model #1
We will have to specify what it is that we are trying to predict and with what data.
- **Target**: Outcome
- **Data**: Game Length, All Tasks Completed, Sabotages Fixed, Murdered, Ejected, Night, Morning, Afternoon, Evening

In [21]:
target_crewmate1_train = crewmate_train_df["Outcome"]
data_crewmate1_train = crewmate_train_df[["Game Length", "All Tasks Completed", "Sabotages Fixed", "Murdered", "Evening"]]

target_crewmate1_test = crewmate_test_df["Outcome"]
data_crewmate1_test = crewmate_test_df[["Game Length", "All Tasks Completed", "Sabotages Fixed", "Murdered", "Evening"]]

features_crewmate1 = data_crewmate1_train.columns

### Decision Tree Classification - Model #1
We perform a decision tree classification in order to assess the validity of our features.

In [22]:
dtc_crewmate1 = tree.DecisionTreeClassifier()
dtc_crewmate1 = dtc_crewmate1.fit(data_crewmate1_train, target_crewmate1_train)
dtc_crewmate1.score(data_crewmate1_test, target_crewmate1_test)

0.5606694560669456

In [23]:
sorted(zip(dtc_crewmate1.feature_importances_, features_crewmate1), reverse=True)

[(0.74447207058485, 'Game Length'),
 (0.09219444102280017, 'Sabotages Fixed'),
 (0.07027560802114202, 'Evening'),
 (0.047330899325364564, 'All Tasks Completed'),
 (0.045726981045843156, 'Murdered')]

In [24]:
dtc_predict_crewmate1_df = pd.DataFrame({
    "Prediction_c1" : dtc_crewmate1.predict(data_crewmate1_test),
    "Actual_c1" : target_crewmate1_test
})
dtc_predict_crewmate1_df["Equals_c1"] = dtc_predict_crewmate1_df["Prediction_c1"].eq(dtc_predict_crewmate1_df["Actual_c1"])
dtc_predict_crewmate1_df

,Prediction_c1,Actual_c1,Equals_c1
0,0,1,False
1,1,1,True
2,0,0,True
3,0,0,True
4,0,0,True
...,...,...,...
234,0,0,True
235,1,0,False
236,1,1,True
237,1,1,True


In [25]:
dtc_predict_crewmate1_df["Equals_c1"].value_counts()

True     134
False    105
Name: Equals_c1, dtype: int64

### Random Forest Classification - Model #1
We perform a random forest classification in order to assess the validity of our features.

In [26]:
rfc_crewmate1 = RandomForestClassifier()
rfc_crewmate1 = rfc_crewmate1.fit(data_crewmate1_train, target_crewmate1_train)
rfc_crewmate1.score(data_crewmate1_test, target_crewmate1_test)

0.5732217573221757

In [27]:
sorted(zip(rfc_crewmate1.feature_importances_, features_crewmate1), reverse=True)

[(0.8627379135792111, 'Game Length'),
 (0.048672939769021414, 'Sabotages Fixed'),
 (0.04823418500271146, 'Murdered'),
 (0.02194502441754402, 'Evening'),
 (0.018409937231512005, 'All Tasks Completed')]

In [28]:
rfc_predict_crewmate1_df = pd.DataFrame({
    "Prediction_c1" : rfc_crewmate1.predict(data_crewmate1_test),
    "Actual_c1" : target_crewmate1_test
})
rfc_predict_crewmate1_df["Equals_c1"] = rfc_predict_crewmate1_df["Prediction_c1"].eq(rfc_predict_crewmate1_df["Actual_c1"])
rfc_predict_crewmate1_df

,Prediction_c1,Actual_c1,Equals_c1
0,0,1,False
1,1,1,True
2,0,0,True
3,0,0,True
4,0,0,True
...,...,...,...
234,0,0,True
235,1,0,False
236,1,1,True
237,1,1,True


In [29]:
rfc_predict_crewmate1_df["Equals_c1"].value_counts()

True     137
False    102
Name: Equals_c1, dtype: int64

### Define Model #2
We will have to specify what it is that we are trying to predict and with what data.
- **Target**: Outcome
- **Data**: Game Length, Task Completed, Sabotages Fixed, Murdered, Ejected, Night, Morning, Afternoon, Evening

In [30]:
target_crewmate2_train = crewmate_train_df["Outcome"]
data_crewmate2_train = crewmate_train_df[["Game Length", "Task Completed", "Sabotages Fixed", "Murdered", "Ejected", "Night", "Morning", "Afternoon", "Evening"]]

target_crewmate2_test = crewmate_test_df["Outcome"]
data_crewmate2_test = crewmate_test_df[["Game Length", "Task Completed", "Sabotages Fixed", "Murdered", "Ejected", "Night", "Morning", "Afternoon", "Evening"]]

features_crewmate2 = data_crewmate2_train.columns

### Decision Tree Classification - Model #2
We perform a decision tree classification in order to assess the validity of our features.

In [31]:
dtc_crewmate2 = tree.DecisionTreeClassifier()
dtc_crewmate2 = dtc_crewmate2.fit(data_crewmate2_train, target_crewmate2_train)
dtc_crewmate2.score(data_crewmate2_test, target_crewmate2_test)

0.5857740585774058

In [32]:
sorted(zip(dtc_crewmate2.feature_importances_, features_crewmate2), reverse=True)

[(0.5863470334582772, 'Game Length'),
 (0.14156604472312254, 'Task Completed'),
 (0.08785460068862458, 'Sabotages Fixed'),
 (0.0435941946884785, 'Murdered'),
 (0.03415640356691169, 'Evening'),
 (0.033394245459794206, 'Afternoon'),
 (0.030266114824405825, 'Morning'),
 (0.02682340522809114, 'Night'),
 (0.015997957362294292, 'Ejected')]

In [33]:
dtc_predict_crewmate2_df = pd.DataFrame({
    "Prediction_c2" : dtc_crewmate2.predict(data_crewmate2_test),
    "Actual_c2" : target_crewmate2_test
})
dtc_predict_crewmate2_df["Equals_c2"] = dtc_predict_crewmate2_df["Prediction_c2"].eq(dtc_predict_crewmate2_df["Actual_c2"])
dtc_predict_crewmate2_df

,Prediction_c2,Actual_c2,Equals_c2
0,1,1,True
1,0,1,False
2,0,0,True
3,0,0,True
4,1,0,False
...,...,...,...
234,1,0,False
235,0,0,True
236,1,1,True
237,1,1,True


In [34]:
dtc_predict_crewmate2_df["Equals_c2"].value_counts()

True     140
False     99
Name: Equals_c2, dtype: int64

### Random Forest Classification - Model #2
We perform a random forest classification in order to assess the validity of our features.

In [35]:
rfc_crewmate2 = RandomForestClassifier()
rfc_crewmate2 = rfc_crewmate2.fit(data_crewmate2_train, target_crewmate2_train)
rfc_crewmate2.score(data_crewmate2_test, target_crewmate2_test)

0.6108786610878661

In [36]:
sorted(zip(rfc_crewmate2.feature_importances_, features_crewmate2), reverse=True)

[(0.6606749577108091, 'Game Length'),
 (0.15355607461497595, 'Task Completed'),
 (0.06673199902640936, 'Sabotages Fixed'),
 (0.051490484649614816, 'Murdered'),
 (0.015902303688008174, 'Ejected'),
 (0.014881948712193435, 'Evening'),
 (0.01412575660146049, 'Afternoon'),
 (0.012348456892232606, 'Night'),
 (0.010288018104296068, 'Morning')]

In [37]:
rfc_predict_crewmate2_df = pd.DataFrame({
    "Prediction_c2" : rfc_crewmate2.predict(data_crewmate2_test),
    "Actual_c2" : target_crewmate2_test
})
rfc_predict_crewmate2_df["Equals_c2"] = rfc_predict_crewmate2_df["Prediction_c2"].eq(rfc_predict_crewmate2_df["Actual_c2"])
rfc_predict_crewmate2_df

,Prediction_c2,Actual_c2,Equals_c2
0,0,1,False
1,1,1,True
2,0,0,True
3,0,0,True
4,0,0,True
...,...,...,...
234,0,0,True
235,0,0,True
236,1,1,True
237,1,1,True


In [38]:
rfc_predict_crewmate2_df["Equals_c2"].value_counts()

True     146
False     93
Name: Equals_c2, dtype: int64